In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from wikipedia2vec import Wikipedia2Vec
from tqdm import tqdm
import pickle

In [2]:
# load wikipedia2vec model
en_w2v = Wikipedia2Vec.load("../model/enwiki_20180420_300d.pkl")
ja_w2v = Wikipedia2Vec.load("../model/jawiki_20180420_300d.pkl")

In [3]:
# load word pair dict
pair_df = pd.read_csv("../data/title_pair.csv")
print(len(pair_df))
pair_df.head()

244517


,ja,en
0,ベルギー,Belgium
1,幸福,Happiness
2,ジョージ・ワシントン,George Washington
3,ジャック・バウアー,Jack Bauer
4,ダグラス・アダムズ,Douglas Adams


In [5]:
# transfer word to vector

en_emb_list = np.empty((0, 300))
ja_emb_list = np.empty((0, 300))
tmp_en_emb_list = np.empty((0, 300))
tmp_ja_emb_list = np.empty((0, 300))

for i, (ja_word, en_word) in tqdm(pair_df.iterrows()):
    try:
        en_emb = en_w2v.get_entity_vector(en_word)
        ja_emb = ja_w2v.get_entity_vector(ja_word)
        
        tmp_en_emb_list = np.concatenate([tmp_en_emb_list, [en_emb]], axis=0)
        tmp_ja_emb_list = np.concatenate([tmp_ja_emb_list, [ja_emb]], axis=0)
    except KeyError:
        pass
    
    if i % 5000 is 0:
        en_emb_list = np.concatenate([en_emb_list, tmp_en_emb_list], axis=0)
        ja_emb_list = np.concatenate([ja_emb_list, tmp_ja_emb_list], axis=0)
        tmp_en_emb_list = np.empty((0, 300))
        tmp_ja_emb_list = np.empty((0, 300))

244517it [26:30, 153.75it/s]


In [32]:
en_emb_list.shape

(108297, 300)

In [6]:
# fit transfer matrix
model = LinearRegression()
model.fit(X=en_emb_list, y=ja_emb_list)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [31]:
# save model
with open("../model/wikipedia2vec_en2ja_mapping.pkl", 'wb') as f:
    pickle.dump(model, f)

In [3]:
# load model
with open("../model/wikipedia2vec_en2ja_mapping.pkl", 'rb') as f:
    model = pickle.load(f)

In [28]:
ja_w2v.most_similar(ja_w2v.get_entity('食品添加物'))

[(<Entity 食品添加物>, 1.0),
 (<Entity 保存料>, 0.63264865),
 (<Word 添加>, 0.6287108),
 (<Entity E番号>, 0.6178505),
 (<Entity 既存添加物>, 0.60397565),
 (<Entity メチルセルロース>, 0.588875),
 (<Word 食品>, 0.5805184),
 (<Entity 固化防止剤>, 0.57792395),
 (<Entity 着色料>, 0.5722214),
 (<Word 乳化剤>, 0.5711883),
 (<Entity 増粘安定剤>, 0.5700469),
 (<Word 香味料>, 0.56336987),
 (<Entity 飼料添加物>, 0.5631246),
 (<Word グルコノデルタラクトン>, 0.56001157),
 (<Entity タンパク加水分解物>, 0.558589),
 (<Word ジブチルヒドロキシトルエン>, 0.5549577),
 (<Entity エチルセルロース>, 0.5544855),
 (<Entity ソルビン酸カリウム>, 0.5510931),
 (<Entity 苦味料>, 0.5496751),
 (<Word ズルチン>, 0.54942423),
 (<Word ジクロルボス>, 0.548301),
 (<Entity フェロシアン化ナトリウム>, 0.54822016),
 (<Entity 合成着色料>, 0.54815215),
 (<Word ブチルヒドロキシアニソール>, 0.545406),
 (<Word アセスルファムカリウム>, 0.544562),
 (<Entity 酸味料>, 0.5443647),
 (<Entity エリソルビン酸>, 0.54435545),
 (<Entity 乳酸カルシウム>, 0.5427019),
 (<Entity 緩衝剤>, 0.54267055),
 (<Entity 酸化防止剤>, 0.54160905),
 (<Entity ポストハーベスト農薬>, 0.53956217),
 (<Entity 殺菌料>, 0.5392771),
 (<Word グアーガム>, 0.5387752

In [26]:
en_w2v.most_similar(en_w2v.get_entity('Food additive'))

[(<Entity Food additive>, 1.0),
 (<Entity E number>, 0.7353947),
 (<Entity International Numbering System for Food Additives>, 0.6838014),
 (<Entity Flavor#Taste>, 0.659413),
 (<Entity Food additive#Categories>, 0.6456771),
 (<Entity Food coloring>, 0.64555603),
 (<Word additives>, 0.64474416),
 (<Entity Acidity regulator>, 0.64245236),
 (<Entity Generally recognized as safe>, 0.62289304),
 (<Entity Joint FAO/WHO Expert Committee on Food Additives>, 0.61937386),
 (<Entity Firming agent>, 0.61532456),
 (<Entity Sorbitan monostearate>, 0.61133474),
 (<Word emulsifier>, 0.6111306),
 (<Entity Colour retention agent>, 0.6066559),
 (<Entity Acidulant>, 0.60639346),
 (<Entity Emulsion#Emulsifiers>, 0.60628015),
 (<Word sorbate>, 0.6037769),
 (<Entity Octyl gallate>, 0.6033475),
 (<Word advantame>, 0.6016131),
 (<Entity Potassium bisulfite>, 0.6006213),
 (<Entity Regulation (EU) 1169/2011>, 0.60043937),
 (<Entity Dodecyl gallate>, 0.6002844),
 (<Entity Sodium erythorbate>, 0.5990106),
 (<Entit

In [31]:
# Test
# input English word
input_word = "Paint"
input_vec = en_w2v.get_entity_vector(input_word)
output_vec = model.predict([input_vec])[0]
ja_w2v.most_similar_by_vector(output_vec)

[(<Entity 油性>, 0.7879044869609684),
 (<Entity 剥離剤>, 0.7772058863300642),
 (<Entity パラフィン#流動パラフィン>, 0.7751721392922368),
 (<Entity 顔料#体質顔料>, 0.7691509748014852),
 (<Word オイルステイン>, 0.7672589115434352),
 (<Entity 顔料#耐性>, 0.7668673835289359),
 (<Entity 顔料#無機顔料>, 0.7663498553361502),
 (<Entity フィキサチーフ>, 0.7608329736755947),
 (<Entity 腐食#.E9.87.91.E5.B1.9E.E3.81.AE.E9.98.B2.E9.A3.9F.E6.96.B9.E6.B3.95>,
  0.7537997872876747),
 (<Entity サーフェイサー>, 0.7535193045426279),
 (<Entity 五酸化アンチモン>, 0.7514707589958423),
 (<Entity ダンマル樹脂>, 0.7431319814637631),
 (<Entity シージングボード>, 0.7426035184461534),
 (<Entity インク#印刷インキ>, 0.7423841679007954),
 (<Entity 保湿剤>, 0.7417185908869653),
 (<Entity マンガン鋼>, 0.7403232065378526),
 (<Word レジンモールド>, 0.7389890668305765),
 (<Word こびりつか>, 0.7388103175320909),
 (<Entity エポキシパテ>, 0.7380682173299541),
 (<Entity 化粧合板>, 0.7368794908515116),
 (<Entity ウォッシュ (絵画技法)>, 0.7365775996073288),
 (<Entity へら#調理器具>, 0.7365066785514887),
 (<Word マイラーバルーン>, 0.7363902670881692),
 (<Word ターポリ